# Emotion Classification from Text
This notebook builds a logistic regression model to classify emotions based on text.

## 1. Install and Import Dependencies

In [ ]:
!pip install kagglehub --quiet
!pip install gradio --quiet

import pandas as pd
import matplotlib.pyplot as plt
import os
import joblib
import kagglehub
import gradio as gr

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


## 2. Download and Load Dataset

In [ ]:
# Download dataset from Kaggle using kagglehub
path = kagglehub.dataset_download("pashupatigupta/emotion-detection-from-text")

# List CSV file
csv_files = [f for f in os.listdir(path) if f.endswith(".csv")]
if not csv_files:
    raise FileNotFoundError("No CSV file found in the dataset.")
csv_path = os.path.join(path, csv_files[0])
print("Loaded CSV:", csv_path)

# Load into pandas
df = pd.read_csv(csv_path)
df.head()


## 3. Preprocessing and Train-Test Split

In [ ]:
# Split dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Extract features and labels
X_train = train_df['content']
y_train = train_df['sentiment']
X_test = test_df['content']
y_test = test_df['sentiment']


## 4. Vectorization and Model Training

In [ ]:
# TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)


## 5. Evaluation

In [ ]:
y_pred = model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


## 6. Save Model and Vectorizer

In [ ]:
joblib.dump(model, 'emotion_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


## 7. Define Prediction Function

In [ ]:
def predict_emotion(comment):
    vec = vectorizer.transform([comment])
    return model.predict(vec)[0]

# Example
print(predict_emotion("Scary lightning and thunder I'm glad it's over now"))


## 8. Launch Gradio Interface

In [ ]:
interface = gr.Interface(
    fn=predict_emotion,
    inputs=gr.Textbox(lines=3, placeholder="Type a comment here..."),
    outputs="text",
    title="Emotion Classifier",
    description="Enter a comment to classify the emotion (e.g. joy, fear, anger, etc.)"
)

interface.launch()
